In [1]:
# entity recognition of sentences
# need to be able to identify entities like People or Organisations or Objects
# need to figure out relationships bw entities

import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")

import pandas as pd
import numpy as np


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# raw_text = "There are 67 bird families living near the mountain. If 32 bird families flew away for winter, how many bird families were left near the mountain?"
raw_text = "If there are 7 bottle caps in a box and Linda puts 7 more bottle caps inside, how many bottle caps are in the box?"
text1 = NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)


7 CARDINAL
Linda PERSON
7 CARDINAL


In [4]:
# challenge: identifies people correctly, but couldn;t identify with birds or "the box"

In [5]:
raw_text = "Ram had 3 balloons. Aditi gave him 2 more. How many does he have now?"
text1 = NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)


3 CARDINAL
2 CARDINAL


Try with Dataset (single Operation)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving singleop.json to singleop (1).json


In [13]:
dset = pd.read_json('/content/singleop.json')
print(dset.columns)

# make dataframe with just questions
dquest = dset['sQuestion'].head(15)
# print(dquest)
dquest = dquest.to_frame()
dquest.rename(columns={'sQuestion': 'questions'}, inplace=True)

print(dquest)

Index(['iIndex', 'lAlignments', 'lEquations', 'lSolutions', 'sQuestion'], dtype='object')
                                            questions
0   If there are 7 bottle caps in a box and Linda ...
1   Jose starts with 7 bottle caps. He gets 2 more...
2   Bridget has 4 Skittles. Henry has 4 Skittles. ...
3   Brenda starts with 7 Skittles. She buys 8 more...
4   There are 4 cards. 3 cards more are added. How...
5   There are 4 marbles. 7 marbles more are added....
6   There are 8 oranges. 8 oranges more are added....
7   Carol collects 2 peanuts. Carol's father gives...
8   There are 9 cards. 4 cards more are added. How...
9   If there are 7 bottle caps in a box and Linda ...
10  Bobby collects 2 blocks. Bobby's father gives ...
11  Peter starts with 8 erasers. Bridget gives Pet...
12  If there are 7 crayons in a box and Gerald put...
13  Michelle has 2 crayons. Janet has 2 crayons. I...
14  Anna collects 6 blocks. Anna's father gives An...


In [8]:
# function to do NER for a raw_text
def ner(raw_text):  
  text1 = NER(raw_text)
  l = []
  for word in text1.ents:
      l.append([word.text,word.label_])
  return l

In [14]:
dquest['NER_TAG'] = dquest['questions'].apply(lambda q: ner(q))

In [15]:
print(dquest)

                                            questions  \
0   If there are 7 bottle caps in a box and Linda ...   
1   Jose starts with 7 bottle caps. He gets 2 more...   
2   Bridget has 4 Skittles. Henry has 4 Skittles. ...   
3   Brenda starts with 7 Skittles. She buys 8 more...   
4   There are 4 cards. 3 cards more are added. How...   
5   There are 4 marbles. 7 marbles more are added....   
6   There are 8 oranges. 8 oranges more are added....   
7   Carol collects 2 peanuts. Carol's father gives...   
8   There are 9 cards. 4 cards more are added. How...   
9   If there are 7 bottle caps in a box and Linda ...   
10  Bobby collects 2 blocks. Bobby's father gives ...   
11  Peter starts with 8 erasers. Bridget gives Pet...   
12  If there are 7 crayons in a box and Gerald put...   
13  Michelle has 2 crayons. Janet has 2 crayons. I...   
14  Anna collects 6 blocks. Anna's father gives An...   

                                              NER_TAG  
0     [[7, CARDINAL], [Linda, P

In [16]:
dquest['NUM_PERSON'] = dquest['NER_TAG'].map(lambda a: len([word[1] for word in a if word[1] == "PERSON"]))
dquest['NUM_NUMBERS'] = dquest['NER_TAG'].map(lambda a: len([word[1] for word in a if word[1] == "CARDINAL"]))
dquest['NUM_ORGS'] = dquest['NER_TAG'].map(lambda a: len([word[1] for word in a if word[1] == "ORG"]))


In [17]:
dquest

,questions,NER_TAG,NUM_PERSON,NUM_NUMBERS,NUM_ORGS
0,If there are 7 bottle caps in a box and Linda ...,"[[7, CARDINAL], [Linda, PERSON], [7, CARDINAL]]",1,2,0
1,Jose starts with 7 bottle caps. He gets 2 more...,"[[Jose, PERSON], [7, CARDINAL], [2, CARDINAL],...",3,2,0
2,Bridget has 4 Skittles. Henry has 4 Skittles. ...,"[[Bridget, ORG], [4, CARDINAL], [Henry, PERSON...",2,2,2
3,Brenda starts with 7 Skittles. She buys 8 more...,"[[Brenda, PERSON], [7, CARDINAL], [8, CARDINAL...",2,2,0
4,There are 4 cards. 3 cards more are added. How...,"[[4, CARDINAL], [3, CARDINAL]]",0,2,0
5,There are 4 marbles. 7 marbles more are added....,"[[4 marbles, MONEY], [7 marbles, QUANTITY]]",0,0,0
6,There are 8 oranges. 8 oranges more are added....,"[[8, CARDINAL], [8, CARDINAL]]",0,2,0
7,Carol collects 2 peanuts. Carol's father gives...,"[[2, CARDINAL], [Carol, PERSON], [Carol 5, PRO...",2,1,0
8,There are 9 cards. 4 cards more are added. How...,"[[9, CARDINAL], [4, CARDINAL]]",0,2,0
9,If there are 7 bottle caps in a box and Linda ...,"[[7, CARDINAL], [Linda, PERSON], [7, CARDINAL]]",1,2,0


# Problem in above things!
Need to build model that can identify PEOPLE + OBJECTS

NER is mis-classifying people as orgs, and objects as people.

HOW TO DO THAT T_T